Combining the fusion.py code with the ComPat stochastic inducer
======================================================



In [ ]:
%matplotlib inline
#notebook
import numpy as np
import fusion
import matplotlib.pyplot as plt
plt.ion()
import scipy.constants  
eV = scipy.constants.eV
import ComPat
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

**fusion.solve_Te** solves

$$\frac{3}{2}\;\frac{\partial}{\partial t}\left(n(\rho,t) T(\rho,t)\right) =
    \nabla_\rho \left[ n(\rho,t) \chi(\rho,t) \nabla_\rho
    (T(\rho,t))\right] + S(\rho, t)$$

with a boundary condition given by $Te_{bc}$ and an initial
uniform temperatore of 1000 eV; the quantities are

- $n(\rho,t)$ characterizes the plasma density

- $\chi(\rho,t)$ characterizes the thermal conductivity

- $S(\rho,t)$ characterizes the source

The geometry of the simulation is characterised by the minor radius
$a_0$, major radius $R_0$ and elongation $E_0$
(while the geometry is solved in the cylindrical approximation, the
actual radius used, $a$, is adjusted on the basis of $a_0$
and $E_0$).

Use the default case as the basis.

In [ ]:
Te, ne, rho, rho_norm, Qe = fusion.solve_Te()

Calculate the flux by differentiation and by integration of the source.

In [ ]:
Gd = -np.gradient(Te, rho) * ne * eV
 
Gi = np.cumsum(Qe *  np.diff(rho).mean() * rho) / (rho + np.diff(rho).mean()/2)
Gi = np.append([Gi[0]/2],(Gi[1:]+Gi[:-1])/2)

plt.figure()
plt.plot( rho, Gd, label='Direct differentiation' )
plt.plot( rho, Gi, label='From source integration' )
plt.legend(loc=0)
plt.xlabel('rho')
plt.ylabel('flux')

Parameters for the ComPat stochastic model
- Starting flux, destination flux and overshoot level all set by the steady state flux at mid radius
- 100 points of which the first 10 are overshoot and the remaining 90 the "steady-state" phase

In [ ]:
s2 = 0.2; s3 = 0.2; d2 = 10; d3 = 10
N1 = 10; N2 = 90; N3 = 30
l1 = np.log10(Gi[Gi.shape[0]//2])
l2 = l1; l3 = l1
x1 = l1; x2 = l1**2

First just calculate what the stochastic flux is, and the derived chi assuming the initial Te.

Note that each block is just a restart from the initial state.

In [ ]:
alpha=0.01

runs = {}
for N in  np.arange(0,1000): 
  runs[N] = {}
  runs[N]['x'], runs[N]['mean'], runs[N]['std'], runs[N]['xrange'], runs[N]['x1'], runs[N]['x2'] = ComPat.randomize(N1, N2, N3, l1, l2, l3, s2, s3, d2, d3, x1, x2, alpha)
  runs[N]['chi'] = 10 ** runs[N]['x1'] / Gd[Gd.shape[0]//2]

plt.figure()
plt.plot(10**np.array([r['x1'] for r in runs.values()]))
plt.xlabel('macro iteration')
plt.ylabel('flux')
plt.plot([0,1000], [10**x1, 10**x1])
plt.title('alpha = %s' % (alpha))

In [ ]:
plt.figure()
plt.hist(np.array([r['chi'] for r in runs.values()]))
plt.xlabel('chi')
plt.ylabel('count')
plt.title('alpha = %s' % (alpha))

Now we will calculate the steady state Te profile for each of the cases

In [ ]:
for N in  np.arange(0,1000): 
  print(N)
  runs[N]['Te'], runs[N]['ne'], runs[N]['rho'], runs[N]['rho_norm'], runs[N]['Qe'] = fusion.solve_Te(chi = runs[N]['chi'], plots=False)

Plot the temperatures for all of the cases

In [ ]:
plt.figure()
for k, v in runs.items():
  plt.plot(v['rho'], v['Te'], label=k, alpha=0.25)
plt.plot(rho, Te, linewidth=3, alpha=0.5, label='REF')
plt.xlabel('rho')
plt.ylabel('Te')
plt.title('alpha = %s' % (alpha))

And now look at the distribution function of the core Te and chi

In [ ]:
Te_runs = np.array([r['Te'] for r in runs.values()])
plt.figure()
plt.hist(Te_runs[:,0], bins=20)
print(Te[0])
print(np.mean(Te_runs, axis=0)[0], np.std(Te_runs, axis=0)[0])
plt.xlabel('Te[0]')
plt.ylabel('count')
plt.title('alpha = %s' % (alpha))

Now consider the approach to steady-state where the initial flux is 10% of the steady-state flux.

In [ ]:
alpha=0.001
l1_c = x1/10
x1_c = x1/10

new_runs = {}
for N in  np.arange(0,1000): 
  new_runs[N] = {}
  new_runs[N]['x'], new_runs[N]['mean'], new_runs[N]['std'], new_runs[N]['xrange'], new_runs[N]['x1'], new_runs[N]['x2'] = ComPat.randomize(N1, N2, N3, l1=l1_c, l2=l2, l3=l3, s2=s2, s3=s3, d2=d2, d3=d3, x1=x1_c, x2=x2, alpha=alpha)
  x1_c = new_runs[N]['x1']
  l1_c = x1_c
  new_runs[N]['chi'] = 10 ** new_runs[N]['x1'] / Gd[Gd.shape[0]//2]

plt.figure()
plt.plot(10**np.array([r['x1'] for r in new_runs.values()]))
plt.xlabel('macro iteration')
plt.ylabel('flux')
plt.plot([0,1000], [10**x1, 10**x1])
plt.title('alpha = %s' % (alpha))

In [ ]:
for N in  np.arange(0,1000): 
  print(N)
  new_runs[N]['Te'], new_runs[N]['ne'], new_runs[N]['rho'], new_runs[N]['rho_norm'], new_runs[N]['Qe'] = fusion.solve_Te(chi = new_runs[N]['chi'], plots=False)

Plot the central Te as a function of macro iteration number.

In [ ]:
Te_runs = np.array([r['Te'] for r in new_runs.values()])
plt.figure()
plt.semilogy(Te_runs[:,0])
plt.xlabel('macro iteration')
plt.ylabel('central Te')
plt.plot([0,1000], [Te[0], Te[0]])
plt.title('alpha = %s' % (alpha))

And now look at the distribution of central Te

In [ ]:
plt.figure()
plt.hist(Te_runs[:,0], bins=20)
plt.xlabel('Te[0]')
plt.ylabel('count')
plt.title('alpha = %s' % (alpha))
print(Te[0])
print(np.mean(Te_runs, axis=0)[0], np.std(Te_runs, axis=0)[0])

In [ ]:
plt.figure()
plt.hist(Te_runs[100:,0], bins=20)
plt.xlabel('Te[0]')
plt.ylabel('count')
plt.title('alpha = %s' % (alpha))
print(Te[0])
print(np.mean(Te_runs[100:], axis=0)[0], np.std(Te_runs[100:], axis=0)[0])